In [4]:
import pandas as pd

In [5]:
df = pd.read_csv(
    "SMSSpamCollection.txt", delimiter="\t", header=None, names=["spam", "text"]
)


In [6]:
sizeData = len(df.index)
trainSize = int(0.5*sizeData)
trainDF = df.sample(trainSize)
testDF = df.drop(trainDF.index)


In [7]:
trainDFSpam = trainDF[trainDF["spam"]=="spam"]
trainDFSpam["text"]=trainDFSpam["text"].str.lower()
dfSplitSpam = trainDFSpam.text.str.split(expand=True).stack().value_counts().to_frame()

trainDFHam = trainDF[trainDF["spam"]=="ham"]
trainDFHam["text"]=trainDFHam["text"].str.lower()
dfSplitHam = trainDFHam.text.str.split(expand=True).stack().value_counts().to_frame()

total = dfSplitSpam.join(dfSplitHam,lsuffix='_spam', rsuffix='_ham')
total.fillna(0.001,inplace=True)
spamTotal=total["count_spam"].sum()
hamTotal = total["count_ham"].sum()
total["vraisemblance"]=(total["count_spam"]/spamTotal)/(total["count_ham"]/hamTotal)

spamProbality = spamTotal/hamTotal

C:\Users\melos\AppData\Local\Temp\ipykernel_6764\3630862678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainDFSpam["text"]=trainDFSpam["text"].str.lower()
C:\Users\melos\AppData\Local\Temp\ipykernel_6764\3630862678.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainDFHam["text"]=trainDFHam["text"].str.lower()


In [8]:
total

,count_spam,count_ham,vraisemblance
to,323,780.000,0.980983
a,191,520.000,0.870128
call,172,108.000,3.772750
your,128,201.000,1.508576
you,125,849.000,0.348783
...,...,...,...
buy,1,26.000,0.091113
msg.,1,1.000,2.368936
had.,1,0.001,2368.936190
accident,1,1.000,2.368936


In [9]:
df = total.sort_values("vraisemblance", ascending=False).head(40)

In [10]:
from matplotlib.axis import YAxis
import plotly.express as px
import numpy as np
violin = total[(total["vraisemblance"] > 0) & (total["vraisemblance"] < 10)]
px.violin(violin, y="vraisemblance")

In [11]:
total["vraisemblance"].min()

np.float64(0.02961170237010027)

In [12]:
def TestPhraseList(phraseList,spamProba=1):
    for word in phraseList:
        vraisemblance = total["vraisemblance"].get(word,"NotFound")
        if vraisemblance!="NotFound":
            spamProba*=vraisemblance
    return(spamProba)
print(TestPhraseList(["hello","call","to","win","your","prize"]))
print(TestPhraseList(["hello","call","me","i","miss","you"]))

def TestPhrase(phrase:str,spamProba=1):
    phraseList=phrase.split()
    return TestPhraseList(phraseList,spamProba=spamProba)


print(TestPhrase("Hi how are you ? I have a wonderful proposition for you ! Gain your wonderful price by clicking on this link"))


2783415.0205895687
0.00010097612405983209
0.0011506616491686302


In [18]:
import pandas as pd

df = pd.read_csv("SMSSpamCollection.txt", delimiter="\t", header=None, names=["spam", "text"])
sizeData = len(df.index)
trainSize = int(0.5 * sizeData)

trainDF = df.sample(trainSize)
testDF = df.drop(trainDF.index)

def preprocess_text(df):
    df["text"] = df["text"].str.lower()
    df["char_count"] = df["text"].apply(len)
    return df

trainDF = preprocess_text(trainDF)

trainDFSpam = trainDF[trainDF["spam"] == "spam"]
trainDFHam = trainDF[trainDF["spam"] == "ham"]

trainDFSpamWords = trainDFSpam.text.str.split(expand=True).stack().value_counts().to_frame(name="count_spam")
trainDFHamWords = trainDFHam.text.str.split(expand=True).stack().value_counts().to_frame(name="count_ham")

total = trainDFSpamWords.join(trainDFHamWords, lsuffix='_spam', rsuffix='_ham').fillna(0.001)
spamTotal = total["count_spam"].sum()
hamTotal = total["count_ham"].sum()

total["vraisemblance"] = (total["count_spam"] / spamTotal) / (total["count_ham"] / hamTotal)
spamProbability = spamTotal / hamTotal

def TestPhraseList(phraseList, char_count, spamProba=1):
    for word in phraseList:
        vraisemblance = total["vraisemblance"].get(word, "NotFound")
        if vraisemblance != "NotFound":
            spamProba *= vraisemblance
    spamProba *= (1 + 0.1 * char_count)
    return spamProba

def TestPhrase(phrase, spamProba=1):
    phraseList = phrase.lower().split()
    char_count = len(phrase)
    return TestPhraseList(phraseList, char_count, spamProba=spamProba)

print(TestPhrase("hello call to win your prize"))
print(TestPhrase("hello call me i miss you"))
print(TestPhrase("Hi how are you? I have a wonderful proposition for you! Gain your wonderful prize by clicking on this link."))


5091764.726374087
0.0005965724853421585
16.80228371277959


In [14]:
testDF["text"]=testDF["text"].str.split()

In [19]:
testDF["result"]=testDF["text"].apply(TestPhrase,spamProba=0.5)
testDF["Finalresult"]=['spam' if x > 1 else 'ham' for x in testDF['result']]
wrong = len(testDF[testDF["Finalresult"]!=testDF["spam"]].index)
FP = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="ham")].index)
FN = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="spam")].index)
VP = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="ham")].index)
VN = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="spam")].index)
wrong/len(testDF.index)
exactitude = (VN+VP)/(VN+FN+FP+VP)
print(f"Exactitude : {exactitude}")
rappel = VP /(VP+FN)
print(f"Rappel : {rappel}")
print(f"Rappel négative: {VN/(VN+FP)}")
precision = VP/(VP+FP)
print(f"Precision : {precision}")
print(f"Précision négative: {VN/(VN+FN)}")
F1Score = (2 * precision * rappel) / (precision + rappel)
print(f"F1-Score : {F1Score}")
print(f"F1-Score Négatif: {(2 * VN/(VN+FN) * VN/VN+FP )/ (VN/(VN+FN) + VN/VN+FP)}")

Exactitude : 0.916367552045944
Rappel : 0.9883198562443846
Rappel négative: 0.6303571428571428
Precision : 0.9140008309098463
Précision négative: 0.9313984168865436
F1-Score : 0.9497086121303692
F1-Score Négatif: 0.9996716549851613
